# Install Library

In [ ]:
# For install Text-to-speech model
!git clone https://github.com/myshell-ai/MeloTTS.git
!cd MeloTTS
!pip install -r MeloTTS/requirements.txt
!pip install unidic
!python -m unidic download

# For install another library we need
!pip install pynput soundfile librosa simpleaudio
!pip install --upgrade transformers bitsandbytes huggingface_hub accelerate

Cloning into 'MeloTTS'...
remote: Enumerating objects: 423, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 423 (delta 169), reused 136 (delta 134), pack-reused 212 (from 1)
Receiving objects: 100% (423/423), 6.04 MiB | 8.79 MiB/s, done.
Resolving deltas: 100% (216/216), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Prep

# Log in to your hugging face

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Load Llama model for generate sentences and feedback

In [ ]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Define the BitsAndBytesConfig with 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

# Load the model with the custom config
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # Pass in the custom BitsAndBytesConfig
    device_map="auto"  # Automatically map model layers to available devices (GPU/CPU)
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create a text-generation pipeline using the quantized model
text_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

# Ask Llama to generate sentences

In [ ]:
# You can experiment by edit messages variable as much as you like until Llama generates the words you want.
messages = [
    {"role": "system", "content": "You are an English speaker who is smart in IELTS. Your goal is to help the user learn speaking by providing a sentence that contains at least one pair of words that sound similar but have different meanings, like 'three' and 'tree' or 'one' and 'want'."},
    {"role": "user", "content": "Can you give me just one sentence to help me learn speaking? Please include words that sound similar but have different meanings and provide just the sentence."},
]

# Generate a response
outputs = text_pipeline(
    messages,
    # we limit max_new_token to 128, if we set it higher then the resulting sentences will be longer
    max_new_tokens=128,
    # The higher the value, the more creative the output will be, but it should be remembered that the risk of incoherence also increases.
    temperature=0.7,
)

text = outputs[0]["generated_text"][-1]['content']

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
text

'After working for three years, he decided to retire and live under the shade of a beautiful tree near the lake.'

# Ask user for recording and then send it to Speech-to-text model

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import soundfile as sf
import io
import IPython.display as ipd
from google.colab import output
import numpy as np
import base64
from IPython.display import HTML, display
import ipywidgets as widgets

# Define the model and processor
model_id = "openai/whisper-large-v3"

model_audio = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
processor = AutoProcessor.from_pretrained(model_id)

# Create the pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model_audio,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)

# Global variables
audio_data = None
audio_result = None

# JavaScript to handle audio recording
js = """
var recordButton = document.getElementById('recordButton');
var audioElement = document.getElementById('audio');
var mediaRecorder;
var audioChunks = [];

recordButton.onclick = function() {
    if (mediaRecorder && mediaRecorder.state === "recording") {
        mediaRecorder.stop();
        recordButton.textContent = 'Start Recording';
        recordButton.style.backgroundColor = '#4CAF50';
    } else {
        navigator.mediaDevices.getUserMedia({ audio: true })
            .then(stream => {
                mediaRecorder = new MediaRecorder(stream);
                mediaRecorder.start();

                recordButton.textContent = 'Stop Recording';
                recordButton.style.backgroundColor = '#f44336';

                mediaRecorder.ondataavailable = function(e) {
                    audioChunks.push(e.data);
                }

                mediaRecorder.onstop = function() {
                    var audioBlob = new Blob(audioChunks, { type: 'audio/wav' });
                    audioElement.src = URL.createObjectURL(audioBlob);
                    var reader = new FileReader();
                    reader.readAsDataURL(audioBlob);
                    reader.onloadend = function() {
                        var base64data = reader.result;
                        document.getElementById('audioData').value = base64data;
                    }
                    audioChunks = [];
                }
            });
    }
};
"""

# HTML for audio recording interface
html = """
<button id="recordButton" style="font-size: 20px; padding: 10px 20px; background-color: #4CAF50; color: white; border: none; cursor: pointer;">Start Recording</button>
<br><br>
<audio id="audio" controls></audio>
<input type="hidden" id="audioData">
"""

# Display the HTML and run the JavaScript
display(HTML(html + f"<script>{js}</script>"))

# Create language dropdown
language_dropdown = widgets.Dropdown(
    options=['english', 'indonesian', 'japanese', 'mandarin'],
    value='english',
    description='Language:',
)
display(language_dropdown)

# Function to save audio as MP3
def save_as_mp3(audio_data, filename="recorded_audio.mp3"):
    with open(filename, "wb") as f:
        f.write(audio_data)
    print(f"Audio saved as {filename}")
    return filename

# Function to process audio and display results
def process_audio(b):
    global audio_result
    audio_data = output.eval_js('document.getElementById("audioData").value')
    if audio_data:
        # Remove the "data:audio/wav;base64," prefix
        audio_data = audio_data.split(',')[1]

        # Decode base64 to bytes
        audio_bytes = base64.b64decode(audio_data)

        # Save audio as MP3
        mp3_filename = save_as_mp3(audio_bytes)

        # Perform speech recognition with selected language
        result = pipe(mp3_filename, generate_kwargs={"language": language_dropdown.value})
        print("Transcription:", result["text"])
        audio_result = result["text"]

        # Create a download link for the MP3 file
        href = f'<a href="data:audio/mp3;base64,{audio_data}" download="{mp3_filename}">Download MP3</a>'
        display(HTML(href))
    else:
        print("No audio recorded yet. Please record audio first.")

# Create process button
print(f"the senteces you must say is \n {text}")
process_button = widgets.Button(description="Process Audio", button_style="info")
process_button.on_click(process_audio)
display(process_button)

Dropdown(description='Language:', options=('english', 'indonesian', 'japanese', 'mandarin'), value='english')

the senteces you must say is 
 After working for three years, he decided to retire and live under the shade of a beautiful tree near the lake.


Button(button_style='info', description='Process Audio', style=ButtonStyle())

Audio saved as recorded_audio.mp3


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcription:  After working for 3 years, she decided to retire and live under the shade of a beautiful tree near the lake.


In [ ]:
audio_result

' After working for 3 years, she decided to retire and live under the shade of a beautiful tree near the lake.'

# Ask llama to generate feedback

In [ ]:
messages = [
    {"role": "system", "content": "You are an English speaker who is smart in IELTS. your goal is to give feedback on the user's pronunciation of the given text"},
    {"role": "user", "content": f"I asked the user to say the following words: {text} \n and what the user said is : {audio_result} \n Please provide feedback on the user's pronunciation results and suggestions for improving the user's speaking skills."},
]

# Generate a response
outputs = text_pipeline(
    messages,
    max_new_tokens=2048,
)

feedback = outputs[0]["generated_text"][-1]['content']

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
feedback

'**Pronunciation Feedback:**\n\n1. **"After working for three years"**: The user said "three" instead of "three years". The correct pronunciation is /θriː ˈjɪərz/ (THREE years). The user should emphasize the word "years" to distinguish it from the number "three".\n\n2. **"he decided to retire"**: The user said "she" instead of "he". The correct pronunciation is /hi/ (he). This is a significant error as it changes the meaning of the sentence.\n\n3. **"live under the shade of a beautiful tree"**: The user\'s pronunciation is clear, but there\'s a slight hesitation between "live" and "under". Try to connect the words smoothly by emphasizing the word "live".\n\n4. **"near the lake"**: The user\'s pronunciation is good, but there\'s a slight emphasis on the word "lake". Try to balance the emphasis between "near" and "lake".\n\n**Suggestions for Improving Speaking Skills:**\n\n1. **Practice with a timer**: Set a timer for 1-2 minutes and try to read the text quickly without hesitating. This 

# Now ask text-to-speech to read the feedback

In [ ]:
import sys
import MeloTTS.melo as melo
sys.modules['melo'] = melo
from MeloTTS.melo.api import TTS

# Speed is adjustable, if you want the model speak faster, increase the value
speed = 0.7

# on this, i ask model to read the sentences too
feedback = 'this is how native speaker say your sentences, ' + text + feedback
model = TTS(language='EN')
speaker_ids = model.hps.data.spk2id

# American accent (the model have several accent)
output_path = 'en-us.wav'
model.tts_to_file(feedback, speaker_ids['EN-US'], output_path, speed=speed)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/content/MeloTTS/melo/download_utils.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any us

 > Text split to sentences.
this is how native speaker say your senteces, After working for three years, he decided to retire and live under the shade of a beautiful tree near the lake. **Pronunciation Feedback:** 1. **After working for three years**: The user said three instead of three years.
The correct pronunciation is /θriː ˈjɪərz/ THREE years. The user should emphasize the word years to distinguish it from the number three. 2. **he decided to retire**: The user said she instead of he. The correct pronunciation is /hi/ he. This is a significant error as it changes the meaning of the sentence.
3. **live under the shade of a beautiful tree**: The user's pronunciation is clear, but there's a slight hesitation between live and under. Try to connect the words smoothly by emphasizing the word live. 4. **near the lake**: The user's pronunciation is good,
but there's a slight emphasis on the word lake. Try to balance the emphasis between near and lake. **Suggestions for Improving Speaking

100%|██████████| 9/9 [00:05<00:00,  1.72it/s]


In [ ]:
from IPython.display import Audio

audio = Audio("/content/en-us.wav")
audio